In [1]:
import numpy as np
import pprint as pp

In [2]:
# THE GIVEN GRAPH
# 1: 2,3,4
# 2: 1,3,5,6
# 3: 1,2,4,5
# 4: 1,3,5,8,9
# 5: 2,3,5,7,8
# 6: 2,7
# 7: 5,6,10
# 8: 4,5,9,11,12,13,16
# 9: 4,8,14
# 10: 7,11,12,13,16
# 11: 8,10,12,13,16
# 12: 8,10,11,14,15
# 13: 8,10,11,15,16
# 14: 9,12,17
# 15: 12,13,17
# 16: 8,10,11,13,17
# 17: 14,15,16


In [3]:
X = [
    [0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
    [1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0],
    [1,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0],
    [1,0,1,0,1,0,0,1,1,0,0,0,0,0,0,0,0],
    [0,1,1,1,0,0,1,1,0,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0],
    [0,0,0,1,1,0,0,0,1,0,1,1,1,0,0,1,0],
    [0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,1,1,1,0,0,1,0],
    [0,0,0,0,0,0,0,1,0,1,0,1,1,0,0,1,0],
    [0,0,0,0,0,0,0,1,0,1,1,0,0,1,1,0,0],
    [0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,1,0],
    [0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1],
    [0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1],
    [0,0,0,0,0,0,0,1,0,1,1,0,1,0,0,0,1],
    [0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0]
]

In [4]:
print(len(X),len(X[0])) # printing dimensions
su = 0
for i in X:
    su += sum(i)

print(su) # printing total number of connections

17 17
70


In [5]:
a = {}
for i in range(len(X)):
    a[i+1] = []
    for j in range(len(X[i])):
        if(X[i][j] == 1):
            a[i+1].append(j+1)
pp.pprint(a)       

{1: [2, 3, 4],
 2: [1, 3, 5, 6],
 3: [1, 2, 4, 5],
 4: [1, 3, 5, 8, 9],
 5: [2, 3, 4, 7, 8],
 6: [2, 7],
 7: [5, 6, 10],
 8: [4, 5, 9, 11, 12, 13, 16],
 9: [4, 8, 14],
 10: [7, 11, 12, 13, 16],
 11: [8, 10, 12, 13, 16],
 12: [8, 10, 11, 14, 15],
 13: [8, 10, 11, 15, 16],
 14: [9, 12, 17],
 15: [12, 13, 17],
 16: [8, 10, 11, 13, 17],
 17: [14, 15, 16]}


In [6]:
x = np.array(X, np.int32)

In [7]:
def paths(graph, v):
    """Generate the maximal cycle-free paths in graph starting at v.
    graph must be a mapping from vertices to collections of
    neighbouring vertices.
    """
    path = [v]                  # path traversed so far
    seen = {v}                  # set of vertices in path
    def search():
        dead_end = True
        for neighbour in graph[path[-1]]:
            if neighbour not in seen:
                dead_end = False
                seen.add(neighbour)
                path.append(neighbour)
                yield from search()
                path.pop()
                seen.remove(neighbour)
        if dead_end:
            yield list(path)
    yield from search()

In [8]:
k = sorted(paths(a,1))
ma = -1
ik = -1
for i in k:
    if i[-1] == 17:
        if len(i) > ma:
            ma = len(i)
            ik = i
print(ma, ik) # Longest path in the given graph

17 [1, 2, 6, 7, 5, 3, 4, 8, 9, 14, 12, 10, 11, 16, 13, 15, 17]


In [9]:
# Checking for mismatched edges
for i in range(len(x)):
    for j in range(len(x[i])):
        if x[i,j] != x.T[i,j]:
            print(i+1,j+1)

In [10]:
y = x
i = 1
while y[0,16] == 0:
    y = np.matmul(y,x)
    i += 1  
    
print(i) # Shortest Path length
    

4


In [9]:
import networkx as nx
import itertools as it
from networkx.algorithms import bipartite

def is_planar(G):
    """
    function checks if graph G has K(5) or K(3,3) as minors,
    returns True /False on planarity and nodes of "bad_minor"
    """
    result=True
    bad_minor=[]
    n=len(G.nodes())
    if n>5:
        for subnodes in it.combinations(G.nodes(),6):
            subG=G.subgraph(subnodes)
            if bipartite.is_bipartite(G):# check if the graph G has a subgraph K(3,3)
                X, Y = bipartite.sets(G)
                if len(X)==3:
                    result=False
                    bad_minor=subnodes
    if n>4 and result:
        for subnodes in it.combinations(G.nodes(),5):
            subG=G.subgraph(subnodes)
            if len(subG.edges())==10:# check if the graph G has a subgraph K(5)
                result=False
                bad_minor=subnodes
    return result,bad_minor

#create random planar graph with n nodes and p probability of growing
n=8
p=0.6
while True:
    G=nx.gnp_random_graph(n,p)
    print(G)
    if is_planar(G)[0]:
        print(G)
        break
# #visualize with pygraphviz
# import matplotlib.pyplot as plt
# nx.draw(G)
# # comment the line above and uncomment one of the 3 lines below (try each of them):
# # nx.draw_random(G)
# # nx.draw_circular(G)
# # nx.draw_spectral(G)
# plt.show()


In [11]:
print(nx.gnp_random_graph(n,p))